In [3]:
import os
import cv2
import kagglehub
import numpy as np
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# # Downloaded latest version
# path = kagglehub.dataset_download("msambare/fer2013")

# print("Path to dataset files:", path)

100%|██████████████████████████████████████| 60.3M/60.3M [00:01<00:00, 63.1MB/s]

Extracting files...


Path to dataset files: /Users/ngrokh/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [6]:
# Set the base directory path
base_dir = "fer2013/versions/1"

# Paths to train and test directories
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Emotion label mapping
emotion_labels = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "neutral": 4,
    "sad": 5,
    "surprise": 6
}

In [8]:
def load_dataset(data_dir, emotion_labels, img_size=(48, 48)):
    images = []
    labels = []
    
    # Loop through each emotion folder
    for emotion, label in emotion_labels.items():
        folder_path = os.path.join(data_dir, emotion)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            # Load the image in grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size)  # Resize to 48x48
                img = img / 255.0  # Normalize pixel values
                images.append(img)
                labels.append(label)
    
    # Convert to numpy arrays
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)  # Add channel dimension
    labels = np.array(labels)
    return images, labels

# Load training and test datasets
X_train, y_train = load_dataset(train_dir, emotion_labels)
X_test, y_test = load_dataset(test_dir, emotion_labels)

print("Training samples:", X_train.shape)
print("Test samples:", X_test.shape)

Training samples: (28709, 48, 48, 1)
Test samples: (7178, 48, 48, 1)


In [9]:
# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=7)
y_val = to_categorical(y_val, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

print("Training samples:", X_train.shape)
print("Validation samples:", X_val.shape)
print("Test samples:", X_test.shape)

Training samples: (22967, 48, 48, 1)
Validation samples: (5742, 48, 48, 1)
Test samples: (7178, 48, 48, 1)


In [10]:
# Define the CNN model
model = Sequential([
    Input(shape=(48, 48, 1)),  # Input shape for FER2013
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(7, activation="softmax")  # 7 output classes
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
# Define Callbacks
checkpoint = ModelCheckpoint("fer2013_model.keras", save_best_only=True, monitor="val_loss", verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

# Train the model
history = model.fit(
    X_train, y_train,  # Training data
    validation_data=(X_val, y_val),  # Validation data
    batch_size=64,  # Adjust batch size if needed
    epochs=30,  # Number of epochs
    callbacks=[checkpoint, early_stopping, reduce_lr],  # Add the callbacks here
    verbose=1  # Display training progress
)

Epoch 1/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 673ms/step - accuracy: 0.2256 - loss: 4.4861   
Epoch 1: val_loss improved from inf to 2.45375, saving model to fer2013_model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 256s 704ms/step - accuracy: 0.2257 - loss: 4.4823 - val_accuracy: 0.2609 - val_loss: 2.4538 - learning_rate: 0.0010
Epoch 2/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 664ms/step - accuracy: 0.2896 - loss: 2.3450  
Epoch 2: val_loss improved from 2.45375 to 1.99425, saving model to fer2013_model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 251s 698ms/step - accuracy: 0.2896 - loss: 2.3447 - val_accuracy: 0.3546 - val_loss: 1.9942 - learning_rate: 0.0010
Epoch 3/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.3432 - loss: 1.9737  
Epoch 3: val_loss improved from 1.99425 to 1.75092, saving model to fer2013_model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 265s 737ms/step - accuracy: 0.3432 - loss: 1.9736 - val_accuracy: 0.3802 - val_loss: 1.7509 - learning_rate: 0.0010
Epoch 4/30
359/359 ━━━━━━━━━━━━━━━

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.5667 - loss: 1.6421
Test Loss: 1.5062
Test Accuracy: 0.6234


In [13]:
model.save("fer2013_model.keras")